In [34]:
import os
import pandas as pd
import pyodbc

import numpy as np 
import matplotlib.pyplot as plt

from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns

Accessing SQL database for the ML model

In [35]:
server = 'gen10-data-fundamentals-22-02-sql-server.database.windows.net'
database = 'guinea-pigs-database'
username = 'gPigs'
password = 'GuineaPigs!'
driver='{SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute("SELECT * FROM NonNormalizedData WHERE [SBA Disaster Number] != '0'")
rows = cursor.fetchall()
c = [column[0] for column in cursor.description]
ma_df = pd.DataFrame([tuple(t) for t in rows], columns = c) 
ma_df

,Approved Amount Content,Approved Amount EIDL,Approved Amount Real Estate,County,Damaged Property Zip Code,Fines and Forfeits,Interest Revenue,Market Value,Property Tax,SBA Disaster Number,SBA EIDL Declaration Number,SBA Physical Declaration Number,State,Timestamp,Total Salaries & Wages,Unemployement,Verified Loss Content,Verified Loss Real Estate,Year,_c0
0,0.0,0.0,0.0,NASSAU,11771.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652819314381,41174,5.9,5000.0,0.0,2013,2651
1,0.0,0.0,0.0,NASSAU,11804.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652819314383,41174,5.9,7624.0,295854.0,2013,2651
2,0.0,0.0,0.0,ROCKINGHAM,3844.0,399,74489,1135.68,247643,ME-00025,12127.0,12126,NEW HAMPSHIRE,1652819290363,560620,6.0,0.0,4900.0,2010,2224
3,0.0,0.0,0.0,ROCKINGHAM,3842.0,399,74489,1135.68,247643,ME-00025,12127.0,12126,NEW HAMPSHIRE,1652819290365,560620,6.0,277112.0,2102162.0,2010,2224
4,59000.0,27900.0,600300.0,PASSAIC,7055.0,568,1162,1277.77,26339,NJ-00023,12781.0,12780,NEW JERSEY,1652819296376,72511,11.0,1739950.0,715233.0,2011,2319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,0.0,0.0,0.0,NASSAU,11569.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652819314328,41174,5.9,24200.0,45563.0,2013,2651
1904,386100.0,32100.0,1030600.0,NASSAU,11569.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652819314371,41174,5.9,519051.0,1476792.0,2013,2651
1905,65000.0,0.0,0.0,NASSAU,11561.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652819314374,41174,5.9,65000.0,23664.0,2013,2651
1906,195400.0,460400.0,0.0,NASSAU,11803.0,351,125,1646.49,21006,NY-00130,13366.0,13365,NEW YORK,1652819314376,41174,5.9,816491.0,100210.0,2013,2651


Defining 'X' and 'y' variables. We are predicting Unemployment rate, so we are using that column as our 'y' value. Our 'X' value will be only the numerical data without the unemployment rate.


Executing the train test split method to split our dataset to be used for training and then testing the model. 

In [36]:
y = ma_df['Unemployement']
X = ma_df.drop(['Unemployement', 'County', 'SBA Disaster Number', 'State', 'Timestamp'], axis=1.)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=0)

In [37]:
model = ElasticNet(alpha=1.0, l1_ratio=0.5,max_iter=10000)
ma_enet_model = model.fit(X_train, y_train)

In [38]:
ma_enet_model.coef_ 

array([ 6.31860405e-07, -9.50627775e-07, -6.37688162e-08, -1.62180680e-05,
       -2.93459954e-05, -2.86804841e-05, -8.33247311e-03,  3.30904484e-06,
        2.56459444e-03,  2.22277592e-07,  7.57702869e-07, -1.47513321e-07,
        6.90444492e-08,  0.00000000e+00, -1.02226795e-03])

In [39]:
ma_enet_model.intercept_

-10.098898879233392

In [40]:
train_prediction = ma_enet_model.predict(X_train)[:10]

train_prediction


array([8.40547965, 7.8616939 , 7.65325722, 8.59385253, 8.21544458,
       8.37677433, 8.45455645, 9.08721549, 8.72412763, 8.41154665])

In [41]:
model_prediction = ma_enet_model.predict(X_test)[:10]

model_prediction

array([ 9.34068064,  5.66438706,  9.52096892, 10.09567697,  5.8297388 ,
        8.413811  ,  7.48169418,  5.92362621,  7.82673618,  6.82939193])

In [42]:
from sklearn.model_selection import GridSearchCV
import numpy as np

In [43]:

clf = GridSearchCV(model, {'alpha': [0.5,1,5,10],'l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},cv=10)
clf.fit(X_train, y_train)
clf.cv_results_



{'mean_fit_time': array([0.01083138, 0.00894403, 0.00822198, 0.00902107, 0.00856202,
        0.00836656, 0.0099647 , 0.00952013, 0.00785861, 0.00939782,
        0.00878499, 0.00850363, 0.00902984, 0.00914803, 0.02440121,
        0.0154851 , 0.00955663, 0.00811977, 0.00857594, 0.00902796,
        0.00832324, 0.00879033, 0.00798125, 0.00844886, 0.00813718,
        0.008777  , 0.00992999, 0.00978401, 0.00969238, 0.00845473,
        0.00981913, 0.00845978, 0.00762305, 0.0077682 , 0.00857756,
        0.00809221]),
 'std_fit_time': array([0.00290714, 0.00120591, 0.00170402, 0.00123223, 0.00113204,
        0.00168716, 0.00064538, 0.00207054, 0.00160677, 0.00325326,
        0.00120928, 0.0013272 , 0.00115661, 0.00237348, 0.01587099,
        0.00710068, 0.0040193 , 0.00130567, 0.00145658, 0.00205067,
        0.00178371, 0.00139416, 0.00127613, 0.00120461, 0.00154538,
        0.00156395, 0.00201657, 0.00106852, 0.00210552, 0.00122098,
        0.00080635, 0.00121076, 0.00147372, 0.00101307, 0.001

In [44]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010831,0.002907,0.002903,0.001142,0.5,0.1,"{'alpha': 0.5, 'l1_ratio': 0.1}",0.308483,0.281474,0.166821,0.297409,0.294997,0.245336,0.205527,0.272319,0.319485,0.308512,0.270036,0.047262,1
1,0.008944,0.001206,0.001940,0.000554,0.5,0.2,"{'alpha': 0.5, 'l1_ratio': 0.2}",0.305730,0.278381,0.163334,0.294457,0.289012,0.241952,0.201061,0.270370,0.315305,0.307009,0.266661,0.047470,27
2,0.008222,0.001704,0.001802,0.000548,0.5,0.3,"{'alpha': 0.5, 'l1_ratio': 0.3}",0.305663,0.278311,0.163361,0.294565,0.289039,0.241943,0.201148,0.270292,0.315243,0.306805,0.266637,0.047429,34
3,0.009021,0.001232,0.002013,0.000110,0.5,0.4,"{'alpha': 0.5, 'l1_ratio': 0.4}",0.305634,0.278265,0.163387,0.294673,0.289066,0.241934,0.201235,0.270286,0.315181,0.306787,0.266645,0.047408,33
4,0.008562,0.001132,0.001953,0.000334,0.5,0.5,"{'alpha': 0.5, 'l1_ratio': 0.5}",0.305606,0.278219,0.163413,0.294780,0.289092,0.241925,0.201322,0.270281,0.315118,0.306768,0.266652,0.047387,30
5,0.008367,0.001687,0.001807,0.000755,0.5,0.6,"{'alpha': 0.5, 'l1_ratio': 0.6}",0.305577,0.278173,0.163439,0.294887,0.289118,0.241916,0.201409,0.270276,0.315056,0.306750,0.266660,0.047366,29
6,0.009965,0.000645,0.002081,0.000453,0.5,0.7,"{'alpha': 0.5, 'l1_ratio': 0.7}",0.305548,0.278127,0.163465,0.294994,0.289145,0.241907,0.201495,0.270270,0.314993,0.306731,0.266667,0.047345,26
7,0.009520,0.002071,0.002089,0.000673,0.5,0.8,"{'alpha': 0.5, 'l1_ratio': 0.8}",0.305518,0.278080,0.163491,0.295101,0.289171,0.241897,0.201582,0.270265,0.314931,0.306713,0.266675,0.047324,25
8,0.007859,0.001607,0.001873,0.000360,0.5,0.9,"{'alpha': 0.5, 'l1_ratio': 0.9}",0.305489,0.278034,0.163517,0.295208,0.289197,0.241888,0.201668,0.270259,0.314868,0.306694,0.266682,0.047303,23
9,0.009398,0.003253,0.001649,0.000482,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",0.305708,0.278366,0.163336,0.294463,0.289014,0.241952,0.201067,0.270332,0.315301,0.306912,0.266645,0.047458,32


In [47]:
alphas = np.logspace(0,1,100)

In [48]:
alphas

array([ 1.        ,  1.02353102,  1.04761575,  1.07226722,  1.09749877,
        1.12332403,  1.149757  ,  1.17681195,  1.20450354,  1.23284674,
        1.26185688,  1.29154967,  1.32194115,  1.35304777,  1.38488637,
        1.41747416,  1.45082878,  1.48496826,  1.51991108,  1.55567614,
        1.59228279,  1.62975083,  1.66810054,  1.70735265,  1.7475284 ,
        1.78864953,  1.83073828,  1.87381742,  1.91791026,  1.96304065,
        2.009233  ,  2.05651231,  2.10490414,  2.15443469,  2.20513074,
        2.25701972,  2.3101297 ,  2.36448941,  2.42012826,  2.47707636,
        2.53536449,  2.59502421,  2.65608778,  2.71858824,  2.7825594 ,
        2.84803587,  2.91505306,  2.98364724,  3.05385551,  3.12571585,
        3.19926714,  3.27454916,  3.35160265,  3.43046929,  3.51119173,
        3.59381366,  3.67837977,  3.76493581,  3.85352859,  3.94420606,
        4.03701726,  4.1320124 ,  4.22924287,  4.32876128,  4.43062146,
        4.53487851,  4.64158883,  4.75081016,  4.86260158,  4.97

In [49]:
clf2 = GridSearchCV(model, {'alpha': alphas,'l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],'normalize':[True,False],},cv=10)
clf2.fit(X_train, y_train)
clf2.cv_results_
df2 = pd.DataFrame(clf2.cv_results_)
df2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,param_normalize,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010094,0.003740,0.002481,0.001483,1.0,0.1,True,"{'alpha': 1.0, 'l1_ratio': 0.1, 'normalize': T...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901
1,0.008472,0.000780,0.002127,0.000260,1.0,0.1,False,"{'alpha': 1.0, 'l1_ratio': 0.1, 'normalize': F...",0.305708,0.278366,...,0.294463,0.289014,0.241952,0.201067,0.270332,0.315301,0.306912,0.266645,0.047458,859
2,0.007354,0.001155,0.001915,0.000431,1.0,0.2,True,"{'alpha': 1.0, 'l1_ratio': 0.2, 'normalize': T...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901
3,0.007384,0.001051,0.002045,0.000419,1.0,0.2,False,"{'alpha': 1.0, 'l1_ratio': 0.2, 'normalize': F...",0.305632,0.278263,...,0.294678,0.289068,0.241934,0.201241,0.270286,0.315176,0.306785,0.266645,0.047406,858
4,0.006842,0.001060,0.001836,0.000508,1.0,0.3,True,"{'alpha': 1.0, 'l1_ratio': 0.3, 'normalize': T...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,0.010713,0.001116,0.002982,0.001760,10.0,0.7,False,"{'alpha': 10.0, 'l1_ratio': 0.7, 'normalize': ...",0.300795,0.271399,...,0.307540,0.291270,0.239956,0.211933,0.268736,0.305799,0.303331,0.266711,0.044849,682
1796,0.008899,0.001476,0.002394,0.000498,10.0,0.8,True,"{'alpha': 10.0, 'l1_ratio': 0.8, 'normalize': ...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901
1797,0.009708,0.001449,0.002142,0.000516,10.0,0.8,False,"{'alpha': 10.0, 'l1_ratio': 0.8, 'normalize': ...",0.299943,0.270305,...,0.309107,0.291348,0.239541,0.213286,0.268371,0.304289,0.302670,0.266554,0.044528,894
1798,0.008849,0.000713,0.002262,0.000430,10.0,0.9,True,"{'alpha': 10.0, 'l1_ratio': 0.9, 'normalize': ...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901


In [50]:
df2.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,param_normalize,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
1753,0.010820,0.002722,0.002340,0.000484,9.545485,0.4,False,"{'alpha': 9.545484566618342, 'l1_ratio': 0.4, ...",0.303259,0.274726,...,0.302022,0.290610,0.241063,0.207275,0.269663,0.310362,0.305170,0.266928,0.045956,1
1735,0.009555,0.001472,0.002278,0.000394,9.326033,0.4,False,"{'alpha': 9.326033468832199, 'l1_ratio': 0.4, ...",0.303323,0.274816,...,0.301857,0.290583,0.241089,0.207137,0.269683,0.310484,0.305216,0.266928,0.045989,2
1575,0.009732,0.001198,0.002326,0.000502,7.564633,0.5,False,"{'alpha': 7.56463327554629, 'l1_ratio': 0.5, '...",0.303294,0.274773,...,0.301937,0.290594,0.241076,0.207198,0.269673,0.310428,0.305195,0.266928,0.045974,3
1771,0.009149,0.001133,0.001944,0.000585,9.7701,0.4,False,"{'alpha': 9.770099572992255, 'l1_ratio': 0.4, ...",0.303194,0.274634,...,0.302191,0.290638,0.241036,0.207415,0.269641,0.310236,0.305123,0.266928,0.045922,4
1593,0.009540,0.001805,0.002275,0.000442,7.742637,0.5,False,"{'alpha': 7.742636826811272, 'l1_ratio': 0.5, ...",0.303230,0.274683,...,0.302104,0.290622,0.241049,0.207338,0.269652,0.310304,0.305149,0.266928,0.045941,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,0.007557,0.001321,0.002246,0.000664,4.641589,0.9,True,"{'alpha': 4.641588833612779, 'l1_ratio': 0.9, ...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901
594,0.008513,0.001103,0.002251,0.000666,2.154435,0.1,True,"{'alpha': 2.154434690031884, 'l1_ratio': 0.1, ...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901
1206,0.009159,0.002215,0.002175,0.000642,4.75081,0.1,True,"{'alpha': 4.7508101621027965, 'l1_ratio': 0.1,...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901
1236,0.008760,0.001242,0.002073,0.000552,4.862602,0.7,True,"{'alpha': 4.862601580065355, 'l1_ratio': 0.7, ...",-0.000050,-0.000356,...,-0.002845,-0.023152,-0.013949,-0.001180,-0.003167,-0.000886,-0.001056,-0.005383,0.007170,901


In [53]:
alphas = np.logspace(0.95,1,50)
alphas

array([ 8.91250938,  8.93347463,  8.95448919,  8.97555318,  8.99666673,
        9.01782994,  9.03904293,  9.06030582,  9.08161873,  9.10298178,
        9.12439508,  9.14585875,  9.16737291,  9.18893768,  9.21055318,
        9.23221952,  9.25393683,  9.27570523,  9.29752483,  9.31939576,
        9.34131814,  9.36329209,  9.38531773,  9.40739518,  9.42952456,
        9.451706  ,  9.47393961,  9.49622553,  9.51856387,  9.54095476,
        9.56339832,  9.58589468,  9.60844395,  9.63104627,  9.65370176,
        9.67641054,  9.69917274,  9.72198848,  9.74485789,  9.7677811 ,
        9.79075823,  9.81378942,  9.83687478,  9.86001444,  9.88320854,
        9.9064572 ,  9.92976054,  9.9531187 ,  9.97653181, 10.        ])

In [54]:
clf3 = GridSearchCV(model, {'alpha': alphas,'l1_ratio': [0.37,0.38,0.39,0.4,0.41,0.42,0.43],},cv=10)
clf3.fit(X_train, y_train)
clf3.cv_results_
df3 = pd.DataFrame(clf3.cv_results_)
df3.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
344,0.010393,0.001774,0.002466,0.000514,10.0,0.38,"{'alpha': 10.0, 'l1_ratio': 0.38}",0.303270,0.274742,0.165127,0.301993,0.290606,0.241067,0.207251,0.269666,0.310383,0.305178,0.266928,0.045961,1
337,0.010532,0.001819,0.002357,0.000387,9.976532,0.38,"{'alpha': 9.976531814691143, 'l1_ratio': 0.38}",0.303277,0.274751,0.165123,0.301976,0.290603,0.241070,0.207237,0.269669,0.310395,0.305183,0.266928,0.045965,2
330,0.009204,0.001180,0.002617,0.000549,9.953119,0.38,"{'alpha': 9.953118704954456, 'l1_ratio': 0.38}",0.303283,0.274760,0.165119,0.301959,0.290600,0.241073,0.207223,0.269671,0.310408,0.305188,0.266928,0.045968,3
323,0.010200,0.001074,0.002477,0.000495,9.929761,0.38,"{'alpha': 9.929760541537563, 'l1_ratio': 0.38}",0.303289,0.274769,0.165116,0.301942,0.290598,0.241075,0.207209,0.269673,0.310420,0.305192,0.266928,0.045971,4
275,0.010810,0.000979,0.002586,0.000476,9.767781,0.39,"{'alpha': 9.76778110089489, 'l1_ratio': 0.39}",0.303265,0.274734,0.165130,0.302008,0.290608,0.241065,0.207264,0.269664,0.310372,0.305174,0.266928,0.045958,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0.011138,0.003616,0.002565,0.000404,9.976532,0.43,"{'alpha': 9.976531814691143, 'l1_ratio': 0.43}",0.302916,0.274246,0.165333,0.302894,0.290748,0.240919,0.208002,0.269547,0.309705,0.304922,0.266923,0.045782,346
14,0.008367,0.001085,0.001897,0.000584,8.954489,0.37,"{'alpha': 8.954489186289914, 'l1_ratio': 0.37}",0.303618,0.275235,0.164913,0.301067,0.290447,0.241209,0.206481,0.269777,0.311057,0.305426,0.266923,0.046145,347
349,0.010008,0.001609,0.002386,0.000457,10.0,0.43,"{'alpha': 10.0, 'l1_ratio': 0.43}",0.302909,0.274236,0.165337,0.302913,0.290751,0.240915,0.208018,0.269545,0.309691,0.304917,0.266923,0.045779,348
7,0.008036,0.001832,0.001866,0.000474,8.933475,0.37,"{'alpha': 8.933474625132913, 'l1_ratio': 0.37}",0.303623,0.275243,0.164910,0.301052,0.290444,0.241211,0.206469,0.269779,0.311068,0.305430,0.266923,0.046148,349


In [55]:
alphas = np.logspace(0.98,1.1,50)
alphas

array([ 9.54992586,  9.60392986,  9.65823925,  9.71285575,  9.7677811 ,
        9.82301705,  9.87856536,  9.93442778,  9.99060611, 10.04710212,
       10.1039176 , 10.16105438, 10.21851425, 10.27629906, 10.33441064,
       10.39285083, 10.4516215 , 10.51072451, 10.57016174, 10.62993508,
       10.69004644, 10.75049772, 10.81129085, 10.87242776, 10.93391039,
       10.9957407 , 11.05792066, 11.12045223, 11.18333742, 11.24657822,
       11.31017664, 11.37413471, 11.43845445, 11.50313791, 11.56818715,
       11.63360424, 11.69939126, 11.7655503 , 11.83208346, 11.89899286,
       11.96628063, 12.03394891, 12.10199984, 12.1704356 , 12.23925835,
       12.30847029, 12.37807362, 12.44807054, 12.5184633 , 12.58925412])

In [58]:
clf4 = GridSearchCV(model, {'alpha': alphas,'l1_ratio': [0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.4,0.41,0.42,0.43],},cv=10)
clf4.fit(X_train, y_train)
clf4.cv_results_
df4 = pd.DataFrame(clf4.cv_results_)
df4.sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
363,0.010876,0.001742,0.003006,0.001411,11.503138,0.33,"{'alpha': 11.503137909836092, 'l1_ratio': 0.33}",0.303266,0.274738,0.165130,0.302000,0.290609,0.241067,0.207261,0.269666,0.310375,0.305176,0.266929,0.045959,1
374,0.011752,0.002347,0.002332,0.000417,11.568187,0.33,"{'alpha': 11.568187152769582, 'l1_ratio': 0.33}",0.303250,0.274716,0.165139,0.302040,0.290615,0.241060,0.207295,0.269661,0.310345,0.305164,0.266929,0.045951,2
352,0.014764,0.003114,0.003523,0.002440,11.438454,0.33,"{'alpha': 11.43845444625509, 'l1_ratio': 0.33}",0.303281,0.274759,0.165120,0.301959,0.290602,0.241073,0.207227,0.269671,0.310405,0.305187,0.266929,0.045967,3
309,0.011778,0.001396,0.002697,0.000459,11.183337,0.34,"{'alpha': 11.183337422167208, 'l1_ratio': 0.34}",0.303263,0.274733,0.165131,0.302008,0.290610,0.241065,0.207268,0.269665,0.310369,0.305173,0.266929,0.045957,4
298,0.020883,0.004971,0.004498,0.001560,11.120452,0.34,"{'alpha': 11.120452233401258, 'l1_ratio': 0.34}",0.303278,0.274755,0.165122,0.301968,0.290603,0.241072,0.207234,0.269670,0.310399,0.305185,0.266929,0.045965,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,0.011196,0.001717,0.002689,0.000568,12.30847,0.43,"{'alpha': 12.308470288172874, 'l1_ratio': 0.43}",0.302152,0.273199,0.165739,0.304701,0.290999,0.240586,0.209524,0.269274,0.308270,0.304360,0.266880,0.045421,546
516,0.015279,0.005683,0.003934,0.001012,12.378074,0.43,"{'alpha': 12.378073615559776, 'l1_ratio': 0.43}",0.302129,0.273168,0.165751,0.304754,0.291005,0.240575,0.209569,0.269265,0.308226,0.304343,0.266879,0.045410,547
527,0.012872,0.002129,0.003051,0.000519,12.448071,0.43,"{'alpha': 12.448070543700462, 'l1_ratio': 0.43}",0.302105,0.273136,0.165763,0.304807,0.291012,0.240565,0.209614,0.269256,0.308183,0.304325,0.266877,0.045399,548
538,0.011192,0.002983,0.002611,0.000701,12.518463,0.43,"{'alpha': 12.518463298372907, 'l1_ratio': 0.43}",0.302081,0.273104,0.165775,0.304860,0.291018,0.240554,0.209659,0.269248,0.308139,0.304308,0.266874,0.045389,549


In [17]:
final_model = ElasticNet(alpha=2.888059, l1_ratio=0.93,max_iter=10000)

In [18]:
final_model.fit(X_train, y_train)

ElasticNet(alpha=2.888059, l1_ratio=0.93, max_iter=10000)

In [19]:
final_model.coef_

array([ 6.16378145e-07, -1.61976824e-06, -1.63205926e-07, -2.41025710e-05,
        7.36932459e-06, -1.32090349e-05, -2.89889074e-03, -2.79725448e-07,
        1.08532395e-04,  4.22760063e-08,  3.88752629e-07, -2.71394658e-07,
        1.50979104e-07,  0.00000000e+00, -3.73080671e-04])

In [20]:
final_model.intercept_

12.652972218797021

In [21]:
finalmodelpredictions = final_model.predict(X_test)
finalmodelpredictions

array([6.73016608, 8.33480704, 6.59051097, ..., 6.56402109, 9.06747554,
       9.66940218])

In [29]:
len(finalmodelpredictions)

1366

In [22]:
(y_test)[:10]

678      9.7
3865     7.9
5005     5.3
4422     8.2
1852     9.2
2797     2.8
863     11.6
4302     9.1
1859     9.2
2407    15.3
Name: Unemployement, dtype: object

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
np.sqrt(mean_squared_error(y_test,finalmodelpredictions))

2.515851563861444

In [26]:
from sklearn.metrics import r2_score

In [27]:
r2_score(y_test,finalmodelpredictions)

0.18424102443808632

In [59]:
f2= ElasticNet(alpha=9.545485, l1_ratio=0.4,max_iter=10000)
f2.fit(X_train, y_train)
f2predictions = f2.predict(X_test)
r2_score(y_test,f2predictions)

0.2587160880010889

In [62]:
f3= ElasticNet(alpha=10, l1_ratio=0.38,max_iter=10000)
f3.fit(X_train, y_train)
f3predictions = f3.predict(X_test)
r2_score(y_test,f3predictions)

0.2587112807466774

In [63]:
f4= ElasticNet(alpha=11.503138, l1_ratio=0.33,max_iter=10000)
f4.fit(X_train, y_train)
f4predictions = f4.predict(X_test)
r2_score(y_test,f4predictions)

0.2587118622989706